In [2]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import subprocess
import os

# Input link YouTube
url = input("Masukkan link YouTube: ")
yt = YouTube(url, on_progress_callback=on_progress)

print(f"\nJudul Video: {yt.title}")
print("\n=== Daftar Stream Tersedia ===")
streams = yt.streams.filter(file_extension="mp4").order_by("resolution").desc()

# Tampilkan daftar dengan nomor
for i, stream in enumerate(streams, start=1):
    if stream.resolution:  # Video
        print(f"{i}. {stream.resolution} {stream.fps}fps - {'Progressive' if stream.is_progressive else 'Adaptive'}")
    else:  # Audio only
        print(f"{i}. Audio {stream.abr}")

# Pilih stream
choice = int(input("\nPilih nomor stream: ")) - 1
stream = streams[choice]

# Input nama file output
filename = input("Masukkan nama file output (tanpa ekstensi): ")

# Tentukan ekstensi
ext = ".mp4" if stream.type == "video" else ".m4a"

# Download video
print(f"\n🎬 Mendownload: {stream}")
video_path = stream.download(filename=filename + ext)

print("\n✅ Video selesai di-download!")

# Kalau adaptive (video tanpa audio), otomatis download audio terbaik + gabungkan
if stream.is_progressive is False and stream.type == "video":
    print("\n🎵 Video tanpa audio, mendownload audio terbaik...")
    audio_stream = yt.streams.filter(only_audio=True, file_extension="mp4").order_by("abr").desc().first()
    audio_path = audio_stream.download(filename=filename + "_audio.m4a")
    
    output_name = filename + "_final.mp4"
    
    print("\n🔄 Menggabungkan video + audio dengan ffmpeg...\n")
    cmd = [
        # Ubah Sesuai PATH ffmpeg di PC/Laptop Kalian
        r"C:\Users\achma\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0-full_build\bin\ffmpeg.exe",
        "-y",
        "-i", video_path,
        "-i", audio_path,
        "-c:v", "copy",
        "-c:a", "aac",
        output_name
    ]
    # tampilkan output ffmpeg (progress bar, dll)
    subprocess.run(cmd, check=True, text=True)
    
    print(f"\n🎉 Selesai! File gabungan: {output_name}")
    
    # Tanya user apakah file sementara mau dihapus
    jawab = input("\n🧹 Hapus file sementara (video tanpa audio & audio)? [Y/n]: ").strip().lower()
    if jawab in ["", "y", "yes"]:
        if os.path.exists(video_path):
            os.remove(video_path)
        if os.path.exists(audio_path):
            os.remove(audio_path)
        print("✅ File sementara sudah dihapus.")
    else:
        print("ℹ️ File sementara tetap disimpan.")
else:
    print("\n🎉 File sudah lengkap (video+audio dalam satu file).")


Masukkan link YouTube:  https://www.youtube.com/watch?v=Kmt71_8fTk8



Judul Video: Belajar Javascript Dasar (Tutorial Lengkap)

=== Daftar Stream Tersedia ===
1. 1080p 60fps - Adaptive
2. 720p 60fps - Adaptive
3. 360p 30fps - Adaptive
4. 360p 30fps - Progressive
5. 144p 30fps - Adaptive



Pilih nomor stream:  1
Masukkan nama file output (tanpa ekstensi):  Belajar Javascript



🎬 Mendownload: <Stream: itag="299" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="avc1.64002a" progressive="False" sabr="False" type="video">
 ↳ |██████████████████████████████████████████████████████████████████| 100.0%
✅ Video selesai di-download!

🎵 Video tanpa audio, mendownload audio terbaik...
 ↳ |██████████████████████████████████████████████████████████████████| 100.0%
🔄 Menggabungkan video + audio dengan ffmpeg...


🎉 Selesai! File gabungan: Belajar Javascript_final.mp4



🧹 Hapus file sementara (video tanpa audio & audio)? [Y/n]:  y


✅ File sementara sudah dihapus.
